In [1]:
import pandas as pd
import plotly.express as px
from dash import Dash, dcc, html, dash_table
from dash.dependencies import Input, Output

In [2]:
 df = pd.read_csv('Employee.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4653 entries, 0 to 4652
Data columns (total 9 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   Education                  4653 non-null   object
 1   JoiningYear                4653 non-null   int64 
 2   City                       4653 non-null   object
 3   PaymentTier                4653 non-null   int64 
 4   Age                        4653 non-null   int64 
 5   Gender                     4653 non-null   object
 6   EverBenched                4653 non-null   object
 7   ExperienceInCurrentDomain  4653 non-null   int64 
 8   LeaveOrNot                 4653 non-null   int64 
dtypes: int64(5), object(4)
memory usage: 327.3+ KB


In [4]:
# Reemplazar los valores de 1 y 0 en la columna 'LeaveOrNot'
df['LeaveOrNot'] = df['LeaveOrNot'].apply(lambda x: 'Alto' if x == 1 else 'Bajo')

In [5]:
# Inicializar la aplicación Dash
app = Dash(__name__)

# Definir las opciones de filtros para los Dropdowns
city_options = [{'label': i, 'value': i} for i in df['City'].unique()]
gender_options = [{'label': i, 'value': i} for i in df['Gender'].unique()]
education_options = [{'label': i, 'value': i} for i in df['Education'].unique()]

In [6]:
# 2. Diseño del Layout de la Aplicación
app.layout = html.Div([ # FONDO GENERAL #F2EFE7
    
    html.H1("Dashboard de Gestión de Talento", style={'textAlign': 'center', 'color': '#333333', 'paddingTop': '10px'}),
    html.Hr(style={'borderTop': '1px solid #cccccc'}),

    # BOTÓN DE EXPORTACIÓN
    html.Div([
        html.Button("Generar Reporte Completo (TXT)", id="export-button", n_clicks=0, 
                    style={'backgroundColor': '#1E8449', 'color': 'white', 'padding': '10px 20px', 'border': 'none', 'borderRadius': '5px', 'cursor': 'pointer', 'fontSize': '16px', 'fontWeight': 'bold'}),
        # Este componente maneja la descarga del archivo generado
        dcc.Download(id="download-report") 
    ], style={'textAlign': 'right', 'marginBottom': '20px', 'paddingRight': '10px'}),
  
    ## Sección de Filtros para Perfiles (Buscador de perfiles para asignación a proyectos)
    html.Div([
        html.H3("Filtros de Perfiles y Asignación", style={'marginTop': '5px'}),
        
        # Contenedor de los Dropdowns y Slider
        html.Div([
            html.Div([
                html.Label("Ciudad"),
                dcc.Dropdown(id='city-filter', options=city_options, multi=True, placeholder="Selecciona Ciudad(es)")
            ], style={'width': '33%', 'display': 'inline-block', 'padding': '10px'}),

            html.Div([
                html.Label("Género"),
                dcc.Dropdown(id='gender-filter', options=gender_options, multi=True, placeholder="Selecciona Género(s)")
            ], style={'width': '33%', 'display': 'inline-block', 'padding': '10px'}),

            html.Div([
                html.Label("Educación"),
                dcc.Dropdown(id='education-filter', options=education_options, multi=True, placeholder="Selecciona Educación(es)")
            ], style={'width': '33%', 'display': 'inline-block', 'padding': '10px'}),

            html.Div([
                html.Label("Rango de Edad"),
                dcc.RangeSlider(id='age-slider', min=df['Age'].min(), max=df['Age'].max(), step=1, value=[df['Age'].min(), df['Age'].max()], marks={str(age): str(age) for age in range(df['Age'].min(), df['Age'].max() + 1, 5)})
            ], style={'width': '95%', 'padding': '10px 10px 20px 10px'}),
            
        ], style={'backgroundColor': '#FFFFFF', 'padding': '10px', 'borderRadius': '8px', 'boxShadow': '2px 2px 10px #e0e0e0'}),

        html.H4("Tabla de Perfiles Filtrados (Según experiencia y dominio)"),
        dash_table.DataTable(
            id='profile-table',
            columns=[{"name": i, "id": i} for i in ['City', 'Age', 'Gender', 'Education', 'ExperienceInCurrentDomain', 'PaymentTier', 'LeaveOrNot']],
            page_action='native',
            page_current=0,
            page_size=10,
            style_table={'overflowX': 'auto', 'backgroundColor': '#FFFFFF'}, 
            style_cell={'textAlign': 'left'}
        ),
    ], style={'marginBottom': '30px'}),

    html.Hr(style={'borderTop': '1px solid #cccccc'}),

    ## Panel de Análisis con Gráficos
    html.H2("Panel de Análisis y Correlación", style={'color': '#333333'}),
    
    # Fila 1: Distribución Demográfica y Correlación
    html.Div([
        # Gráfico 1: Distribución Demográfica (Género por Ciudad)
        dcc.Graph(id='demographic-graph', style={'width': '50%', 'display': 'inline-block', 'boxShadow': '2px 2px 10px #e0e0e0', 'marginBottom': '20px'}),

        # Gráfico 2: Correlación entre Nivel de Pago (PaymentTier) y Experiencia
        dcc.Graph(id='correlation-graph', style={'width': '50%', 'display': 'inline-block', 'boxShadow': '2px 2px 10px #e0e0e0', 'marginBottom': '20px'}),
    ], style={'marginBottom': '10px'}),

    # Fila 2: Experiencia vs Riesgo (Strip Plot) y Historial "Benched"
    html.Div([
        # Gráfico 3: Experiencia vs Riesgo de Abandono (Strip Plot/Jitter)
        dcc.Graph(id='experience-risk-graph', style={'width': '50%', 'display': 'inline-block', 'boxShadow': '2px 2px 10px #e0e0e0'}),

        # Gráfico 4: Historial de Empleados Benched
        dcc.Graph(id='benched-history-graph', style={'width': '50%', 'display': 'inline-block', 'boxShadow': '2px 2px 10px #e0e0e0'}),
    ], style={'marginBottom': '30px'}),

    html.Hr(style={'borderTop': '1px solid #cccccc'}),

    ## Exportación de Reportes Tabulares
    html.H2("Reportes para Exportación (Diversidad, Rotación y Talento)", style={'color': '#333333'}),
    
    html.H4("Reporte de Diversidad (Educación y Género)"),
    dash_table.DataTable(
        id='diversity-report-table',
        style_table={'overflowX': 'auto', 'backgroundColor': '#FFFFFF'},
        style_cell={'textAlign': 'center'}
    ),
    html.Br(),

    html.H4("Reporte de Rotación (Riesgo de Abandono por Ciudad)"),
    dash_table.DataTable(
        id='turnover-report-table',
        style_table={'overflowX': 'auto', 'backgroundColor': '#FFFFFF'},
        style_cell={'textAlign': 'center'}
    ),
    html.Br(),

], style={'backgroundColor': '#F2EFE7', 'padding': '20px 40px', 'fontFamily': 'Arial, sans-serif'})

In [7]:
# 3. Callbacks (Lógica de Interacción)
# Callback 1: Tabla de Perfiles Filtrados (Buscador)
@app.callback(
    Output('profile-table', 'data'),
    [
        Input('city-filter', 'value'),
        Input('gender-filter', 'value'),
        Input('education-filter', 'value'),
        Input('age-slider', 'value')
    ]
)
def update_profile_table(cities, genders, educations, age_range):
    filtered_df = df.copy() 
    if cities: filtered_df = filtered_df[filtered_df['City'].isin(cities)]
    if genders: filtered_df = filtered_df[filtered_df['Gender'].isin(genders)]
    if educations: filtered_df = filtered_df[filtered_df['Education'].isin(educations)]
    if age_range:
        min_age, max_age = age_range
        filtered_df = filtered_df[(filtered_df['Age'] >= min_age) & (filtered_df['Age'] <= max_age)]
    return filtered_df[['City', 'Age', 'Gender', 'Education', 'ExperienceInCurrentDomain', 'PaymentTier', 'JoiningYear', 'EverBenched', 'LeaveOrNot']].to_dict('records')

# Callback 2: Gráfico de Distribución Demográfica
@app.callback(
    Output('demographic-graph', 'figure'),
    [Input('profile-table', 'data')] 
)
def update_demographic_graph(data):
    dff = pd.DataFrame(data)
    if dff.empty: return {}
    # Gráfico de barras: Género y Ciudad (Distribución demográfica)
    fig = px.histogram(dff, x='City', color='Gender', title='Distribución Demográfica (Género por Ciudad)', barmode='group')
    fig.update_layout(plot_bgcolor='#FFFFFF', paper_bgcolor='#FFFFFF', margin=dict(t=30, b=30, l=30, r=30))
    return fig

# Callback 3: Correlación entre Experiencia y Pago (HEATMAP)
@app.callback(
    Output('correlation-graph', 'figure'),
    [Input('profile-table', 'data')] 
)
def update_correlation_graph(data):
    dff = pd.DataFrame(data)
    if dff.empty: 
        return {}
    
    # 1. Crear una tabla de contingencia (frecuencia de conteo)
    # Utilizamos 'ExperienceInCurrentDomain' en el índice (Y) y 'PaymentTier' en las columnas (X)
    correlation_data = dff.groupby(['ExperienceInCurrentDomain', 'PaymentTier']).size().reset_index(name='Count')
    
    # 2. Generar el Heatmap
    fig = px.density_heatmap(
        correlation_data,
        x='PaymentTier',
        y='ExperienceInCurrentDomain',
        z='Count', # La intensidad del color (Z) es el conteo de empleados
        title='Correlación: Frecuencia de Empleados por Nivel de Pago y Experiencia',
        histfunc=None, # Ya hemos contado los datos
        color_continuous_scale=px.colors.sequential.Plasma # Puedes elegir cualquier escala de color
    )

    # 3. Ajustar los ejes para mejorar la presentación de las categorías ordinales
    # Aseguramos que los niveles de pago se muestren discretamente (1, 2, 3)
    fig.update_xaxes(
        type='category', 
        title='Nivel de Pago',
        categoryorder='array', 
        categoryarray=[1, 2, 3]
    )
    
    # Aseguramos que la experiencia se muestre de forma discreta
    fig.update_yaxes(
        type='category', 
        title='Años de Experiencia en Dominio Actual',
        categoryorder='array',
        categoryarray=sorted(dff['ExperienceInCurrentDomain'].unique(), reverse=True) # Experiencia descendente
    )

    # Estilo de fondo blanco
    fig.update_layout(plot_bgcolor='#FFFFFF', paper_bgcolor='#FFFFFF', margin=dict(t=30, b=30, l=30, r=30))
    return fig

# Callback 4: Gráfico de Experiencia y Riesgo (VUELVE AL BOXPLOT)
@app.callback(
    Output('experience-risk-graph', 'figure'),
    [Input('profile-table', 'data')]
)
def update_experience_risk_graph(data):
    dff = pd.DataFrame(data)
    if dff.empty: 
        return {}

    # Boxplot para ExperienciaInCurrentDomain vs. LeaveOrNot
    fig = px.box(
        dff, 
        x='LeaveOrNot', 
        y='ExperienceInCurrentDomain', 
        color='LeaveOrNot',
        title='Experiencia vs. Riesgo de Abandono (Boxplot)'
    )
    
    
    fig.update_layout(plot_bgcolor='#FFFFFF', paper_bgcolor='#FFFFFF', margin=dict(t=30, b=30, l=30, r=30))
    return fig

# Callback 5: Historial de Empleados "Benched" (NUEVO)
@app.callback(
    Output('benched-history-graph', 'figure'),
    [Input('profile-table', 'data')] 
)
def update_benched_history_graph(data):
    dff = pd.DataFrame(data)
    if dff.empty: return {}
    
    # Agrupamos por si estuvo "benched" y por año de ingreso
    benched_df = dff.groupby(['JoiningYear', 'EverBenched']).size().reset_index(name='Count')
    
    # Gráfico de barras apiladas o líneas para mostrar tendencia
    fig = px.bar(
        benched_df, 
        x='JoiningYear', 
        y='Count', 
        color='EverBenched',
        title='Historial de Empleados en "Benched" por Año de Ingreso',
        barmode='group'
    )
    fig.update_layout(plot_bgcolor='#FFFFFF', paper_bgcolor='#FFFFFF', margin=dict(t=30, b=30, l=30, r=30))
    return fig


# Callback 6: Tabla de Reporte de Diversidad
@app.callback(
    Output('diversity-report-table', 'data'),
    [Input('profile-table', 'data')] 
)
def update_diversity_report(data):
    diversity_table = df.groupby(['Education', 'Gender']).size().reset_index(name='Total')
    total_row = pd.Series({'Education': 'Total General', 'Gender': '', 'Total': diversity_table['Total'].sum()})
    diversity_table = pd.concat([diversity_table, total_row.to_frame().T], ignore_index=True)
    return diversity_table.to_dict('records')

# Callback 7: Tabla de Reporte de Rotación
@app.callback(
    Output('turnover-report-table', 'data'),
    [Input('profile-table', 'data')]
)
def update_turnover_report(data):
    turnover_table = df.groupby(['City', 'LeaveOrNot']).size().reset_index(name='Conteo')
    city_total = turnover_table.groupby('City')['Conteo'].transform('sum')
    turnover_table['Porcentaje'] = (turnover_table['Conteo'] / city_total * 100).round(2).astype(str) + '%'
    return turnover_table.to_dict('records')

# Callback 8: Lógica de Exportación del Reporte Completo (Requiere kaleido)
@app.callback(
    Output("download-report", "data"),
    [Input("export-button", "n_clicks")],
    [
        # Recibir las variables de filtro para aplicar el estado actual
        Input('city-filter', 'value'),
        Input('gender-filter', 'value'),
        Input('education-filter', 'value'),
        Input('age-slider', 'value'),
        # Podemos incluir otros Inputs si se necesitan para la lógica de reporte
    ],
    prevent_initial_call=True,
)
def generate_full_report(n_clicks, cities, genders, educations, age_range):
    # Esta función debe hacer:
    # 1. Aplicar los filtros (cities, genders, etc.) al DataFrame df.
    # 2. Re-generar las 4 figuras (demographic-graph, correlation-graph, etc.) con esos datos filtrados.
    # 3. Usar fig.write_image() (REQUIERE KALEIDO) para guardar las figuras como PNG.
    # 4. Compilar una plantilla HTML que contenga:
    #    - La lista de filtros aplicados.
    #    - Las tablas de reportes (diversity, turnover) en formato HTML.
    #    - Las imágenes PNG de los 4 gráficos.
    # 5. Generar un archivo PDF a partir del HTML (o simplemente un archivo HTML/ZIP).
    # 6. Devolver el archivo binario a dcc.Download.
    
    # Dado que la implementación es compleja, aquí solo tienes la estructura.
    # Para fines de demostración, devolvemos un archivo de texto simple:
    
    if n_clicks > 0:
        report_content = "--- Reporte de Gestión de Talento ---\n"
        report_content += f"Filtros Aplicados:\n"
        report_content += f"  Ciudades: {cities}\n"
        report_content += f"  Edades: {age_range}\n"
        # Aquí iría toda la lógica de compilación de gráficos y tablas
        
        return dcc.send_string(report_content, "reporte_talento_filtrado.txt")
    
    return None

In [8]:
# 4. Ejecutar la aplicación
if __name__ == '__main__':
    # CORRECCIÓN DE ERROR: Se reemplaza app.run_server por app.run
    app.run(debug=True)